# 🔬 Activity Cliff Calculator

**A Python tool to identify and analyze activity cliffs** — pairs of structurally similar compounds that show large potency differences.

---

## 🔍 Overview

`Activity Cliff Calculator` processes a dataset of bioactive molecules and automatically detects *activity cliffs* based on:
- Differences in biological activity (**ΔpIC50**)  
- Chemical similarity (**Tanimoto coefficient**)

The program uses modern **RDKit** fingerprint generators (Morgan, Feature Morgan, MACCS, RDK, AtomPair, Torsion, Pattern) to compute molecular similarity, and provides both **interactive visualization** and **Excel export** options.

---

The program needs pandas, numpy, rdkit, xlrd, openpyxl, nbformat and plotly.express libraries

Example on an input file https://www.mdpi.com/article/10.3390/ijms23010259/s1 from Macip G, Garcia-Segura P, Mestres-Truyol J, Saldivar-Espinoza B, Pujadas G, Garcia-Vallvé S. A Review of the Current Landscape of SARS-CoV-2 Main Protease Inhibitors: Have We Hit the Bullseye Yet? Int J Mol Sci. 2021 Dec 27;23(1):259. doi: 10.3390/ijms23010259

In [1]:
from activity_cliffs_utils import (
    fp_as_bitvect, generate_pairs,
    export_activity_cliffs_to_excel, show_molecule_table, mol_to_image_bytes, smiles_to_svg
)
from rdkit import Chem
import pandas as pd
import plotly.express as px

Compute activity Cliffs (pairs or groups of structurally similar compounds that are active against the same target but have large differences in potency) from an xls file containing a set of molecules. The columns "standardize_smiles", "PubMedID" and "pIC50" are mandatory. 
The program uses rdkit to compute Morgan fingerprints of each molecule. When the PubMedID is the same, for each pair of molecules a disparity value is calculated as disparity = pIC50_diff / (1 - tanimoto).

The program needs pandas, numpy, rdkit, xlrd, openpyxl, nbformat and plotly.express libraries

Example on an input file https://www.mdpi.com/article/10.3390/ijms23010259/s1 from Macip G, Garcia-Segura P, Mestres-Truyol J, Saldivar-Espinoza B, Pujadas G, Garcia-Vallvé S. A Review of the Current Landscape of SARS-CoV-2 Main Protease Inhibitors: Have We Hit the Bullseye Yet? Int J Mol Sci. 2021 Dec 27;23(1):259. doi: 10.3390/ijms23010259

In [2]:
# ---------- Main execution ----------

# Define which fingerprints you want to calculate
fp_types = ["morgan", "feature_morgan", "rdk", "maccs", "pattern", "atompair", "torsion"]
N_BITS = 2048
RADIUS = 2

# Read input data
df = pd.read_excel("M-pro_Inhibitors.xls")

# Compute fingerprints for all types
for fp_type in fp_types:
    col_name = f"fp_{fp_type}"
    print(f"Computing {fp_type} fingerprints...")
    df[col_name] = df["standardize_smiles"].apply(
        lambda x: fp_as_bitvect(Chem.MolFromSmiles(x), fp_type=fp_type, n_bits=N_BITS, radius=RADIUS)
        if Chem.MolFromSmiles(x) else None
    )

# Generate pairs and calculate disparities
fp_col= "feature_morgan" # Choose between "morgan", "feature_morgan", "rdk", "maccs", "pattern", "atompair", "torsion"
fp_col = f"fp_{fp_col}"
result_df = generate_pairs(df, group_col="PubMedID", fp_col=fp_col)
result_df = result_df.sort_values(by="Disparity", ascending=False)

# Create the interactive chart
fig = px.scatter(
    result_df,
    x='Tanimoto',
    y='pIC50_diff',
    title='Activity difference vs Tanimoto Similarity',
    labels={'Tanimoto': f'Tanimoto Similarity ({fp_col})', 'pIC50_diff': 'Activity difference (ΔpIC50)'},
    hover_data=['Compound1', 'Compound2', 'Disparity']
)
# Adjust the chart dimensions
fig.update_layout(
    height=500  # You can change this value to adjust the height
)
# Show the graph
fig.show()

Computing morgan fingerprints...
Computing feature_morgan fingerprints...
Computing rdk fingerprints...
Computing maccs fingerprints...
Computing pattern fingerprints...
Computing atompair fingerprints...
Computing torsion fingerprints...


In [3]:
# ---------- Filtering section ----------
tanimoto_min = 0.8          # set to None if not used
disparity_min = None         # set to None if not used
activity_diff_min = 1.5    # set to None if not used

# Image export options
image_size_excel = 120
image_size_preview = (150, 150)


# Build dynamic filter mask
mask = pd.Series(True, index=result_df.index)

if tanimoto_min is not None:
    mask &= result_df['Tanimoto'] >= tanimoto_min

if disparity_min is not None:
    mask &= result_df['Disparity'] >= disparity_min

if activity_diff_min is not None:
    mask &= result_df['pIC50_diff'] >= activity_diff_min

filtered_df = result_df[mask].copy()
print(f"✅ Pairs remaining after filtering: {len(filtered_df):,}")

# Generate PNG images (for Excel export)
filtered_df["Mol1_img"] = filtered_df["SMILES1"].apply(
    lambda s: mol_to_image_bytes(s, image_size=(image_size_excel, image_size_excel))
)
filtered_df["Mol2_img"] = filtered_df["SMILES2"].apply(
    lambda s: mol_to_image_bytes(s, image_size=(image_size_excel, image_size_excel))
)

# Generate inline SVG images (for notebook or Visual Studio Code display)
filtered_df["Mol1_svg"] = filtered_df["SMILES1"].apply(
    lambda s: smiles_to_svg(s, size=image_size_preview)
)
filtered_df["Mol2_svg"] = filtered_df["SMILES2"].apply(
    lambda s: smiles_to_svg(s, size=image_size_preview)
)

# ---------- Excel export ----------
output_xlsx = f"disparity_results_feature_morgan_filtered.xlsx"
export_activity_cliffs_to_excel(filtered_df, output_xlsx,image_size=image_size_excel)

# ----------Display Preview Table ----------
show_molecule_table(filtered_df, max_rows=20, img_size=image_size_preview)


✅ Pairs remaining after filtering: 19
✅ Excel file saved with molecule images (120px): disparity_results_feature_morgan_filtered.xlsx
🧪 Displaying all 19 rows with molecule drawings.
